In [1]:
# Import libraries
import tensorflow as tf

In [2]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [3]:
# Define the teacher model (a larger, more complex neural network)
class TeacherModel(tf.keras.Model):
    def __init__(self):
        super(TeacherModel, self).__init__()
        # Define the layers of the teacher model
        self.layer1 = tf.keras.layers.Dense(512, input_shape=(784,), activation='relu')
        self.layer2 = tf.keras.layers.Dense(256, activation='relu')
        self.layer3 = tf.keras.layers.Dense(128, activation='relu')
        self.layer4 = tf.keras.layers.Dense(64, activation='relu')
        self.last   = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        # Define the forward pass of the teacher model
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return self.last(x)


In [4]:
# Define the student model (a smaller, more efficient neural network)
class StudentModel(tf.keras.Model):
    def __init__(self):
        super(StudentModel, self).__init__()
        # Define the layers of the student model
        self.layer1 = tf.keras.layers.Dense(512, input_shape=(784,), activation='relu')
        self.layer2 = tf.keras.layers.Dense(256, activation='relu')
        self.last = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        # Define the forward pass of the student model
        x = self.layer1(x)
        x = self.layer2(x)
        return self.last(x)

In [5]:
# Load the teacher model 
teacher_model = TeacherModel()

# Define the loss function and optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [6]:
# Train the teacher model
teacher_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
teacher_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.2089 - accuracy: 0.9369 - val_loss: 0.1142 - val_accuracy: 0.9659
Epoch 2/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0945 - accuracy: 0.9715 - val_loss: 0.0837 - val_accuracy: 0.9746
Epoch 3/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0676 - accuracy: 0.9797 - val_loss: 0.0764 - val_accuracy: 0.9787
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0517 - accuracy: 0.9840 - val_loss: 0.0945 - val_accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0443 - accuracy: 0.9862 - val_loss: 0.1045 - val_accuracy: 0.9741


In [7]:
# Load the student model
student_model = StudentModel()

# Freeze the teacher model layers
for layer in teacher_model.layers:
    layer.trainable = False

In [8]:
# Define the distillation loss function
temp = 5
def distillation_loss(y_true, y_pred):
    y_true = tf.nn.softmax(y_true / temp)
    y_pred = tf.nn.softmax(y_pred / temp)
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))

In [9]:
# Train the student model
student_model.compile(optimizer=optimizer, loss=distillation_loss, metrics=['accuracy'])
student_model.fit(x_train, teacher_model.predict(x_train), epochs=5, batch_size=32, validation_data=(x_test, y_test))

1875/1875 [==============================] - 4s 2ms/step
Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 2.3008 - accuracy: 0.9372 - val_loss: 2.3007 - val_accuracy: 0.9604
Epoch 2/5
1875/1875 [==============================] - 10s 6ms/step - loss: 2.3007 - accuracy: 0.9711 - val_loss: 2.3007 - val_accuracy: 0.9687
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3007 - accuracy: 0.9776 - val_loss: 2.3007 - val_accuracy: 0.9690
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3007 - accuracy: 0.9811 - val_loss: 2.3007 - val_accuracy: 0.9738
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3007 - accuracy: 0.9840 - val_loss: 2.3007 - val_accuracy: 0.9725


In [10]:
# Evaluate the student model
test_loss, test_acc = student_model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 2.3007 - accuracy: 0.9725
Test accuracy: 0.9725000262260437
